Tutorial : https://www.youtube.com/watch?v=hUXQm46TAKc <br/>
Full View : https://github.com/wandb/examples/blob/master/colabs/pytorch-lightning/Supercharge_your_Training_with_Pytorch_Lightning_%2B_Weights_%26_Biases.ipynb

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
%%capture
! pip install pytorch-lightning
! pip install wandb

In [3]:
# numpy for non-GPU array math
import numpy as np

# 🍦 Vanilla PyTorch
import torch
from torch.nn import functional as F
from torch import nn
from torch.utils.data import DataLoader, random_split
from torch.optim import Adam

# 👀 Torchvision for CV
from torchvision.datasets import MNIST
from torchvision import transforms

In [4]:
# ⚡ PyTorch Lightning
import pytorch_lightning as pl
pl.seed_everything(hash("setting random seeds") % 2**32 - 1)

# 🏋️‍♀️ Weights & Biases
import wandb

# ⚡ 🤝 🏋️‍♀️
from pytorch_lightning.loggers import WandbLogger

# We need create an account first
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [35]:
class LitMLP(pl.LightningModule):

    def __init__(self, in_dims, n_classes=10,
                 n_layer_1=128, n_layer_2=256, lr=1e-4):
        super().__init__()

        # we flatten the input Tensors and pass them through an MLP
        self.layer_1 = nn.Linear(np.prod(in_dims), n_layer_1)
        self.layer_2 = nn.Linear(n_layer_1, n_layer_2)
        self.layer_3 = nn.Linear(n_layer_2, n_classes)

        # log hyperparameters
        self.save_hyperparameters()

        # compute the accuracy -- no need to roll your own!
        self.train_acc = pl.metrics.Accuracy()
        self.valid_acc = pl.metrics.Accuracy()
        self.test_acc = pl.metrics.Accuracy()

    def forward(self, x):
        """
        Defines a forward pass using the Stem-Learner-Task
        design pattern from Deep Learning Design Patterns:
        https://www.manning.com/books/deep-learning-design-patterns
        """
        batch_size, *dims = x.size()

        # stem: flatten
        x = x.view(batch_size, -1)

        # learner: two fully-connected layers
        x = F.relu(self.layer_1(x))
        x = F.relu(self.layer_2(x))
        
        # task: compute class logits
        x = self.layer_3(x)
        x = F.log_softmax(x, dim=1)

        return x

    # convenient method to get the loss on a batch
    def loss(self, xs, ys):
        logits = self(xs)  # this calls self.forward
        loss = F.nll_loss(logits, ys)
        return logits, loss

    def training_step(self, batch, batch_idx):
      xs, ys = batch
      logits, loss = self.loss(xs, ys)

      # logging metrics we calculated by hand
      self.log('train/loss', loss, on_epoch=True)
      # logging a pl.Metric
      self.train_acc(logits, ys)
      self.log('train/acc', self.train_acc, on_epoch=True)
      
      return loss

    def configure_optimizers(self):
      return torch.optim.Adam(self.parameters(), lr=self.hparams["lr"])

    def test_step(self, batch, batch_idx):
      xs, ys = batch
      logits, loss = self.loss(xs, ys)
      self.test_acc(logits, ys)
      self.log("test/loss_epoch", loss, on_step=False, on_epoch=True)
      self.log("test/acc_epoch", self.test_acc, on_step=False, on_epoch=True)
   
    def test_epoch_end(self, test_step_outputs):  # args are defined as part of pl API
      dummy_input = torch.zeros(self.hparams["in_dims"], device=self.device)
      model_filename = "model_final.onnx"
      torch.onnx.export(self, dummy_input, model_filename)
      wandb.save(model_filename)
   
    def validation_step(self, batch, batch_idx):
      xs, ys = batch
      logits, loss = self.loss(xs, ys)
      preds = torch.argmax(logits, 1)
      self.valid_acc(logits, ys)

      self.log("valid/loss_epoch", loss)  # default on val/test is on_epoch only
      self.log('valid/acc_epoch', self.valid_acc)
        
      return logits


    def validation_epoch_end(self, validation_step_outputs):
      dummy_input = torch.zeros(self.hparams["in_dims"], device=self.device)
      model_filename = f"model_{str(self.global_step).zfill(5)}.onnx"
      torch.onnx.export(self, dummy_input, model_filename)
      wandb.save(model_filename)

      flattened_logits = torch.flatten(torch.cat(validation_step_outputs))
      self.logger.experiment.log(
          {"valid/logits": wandb.Histogram(flattened_logits.to("cpu")),
          "global_step": self.global_step})

Extra fancy loggings

In [36]:
class ImagePredictionLogger(pl.Callback):
    def __init__(self, val_samples, num_samples=32):
        super().__init__()
        self.val_imgs, self.val_labels = val_samples
        self.val_imgs = self.val_imgs[:num_samples]
        self.val_labels = self.val_labels[:num_samples]
          
    def on_validation_epoch_end(self, trainer, pl_module):
        val_imgs = self.val_imgs.to(device=pl_module.device)

        logits = pl_module(val_imgs)
        preds = torch.argmax(logits, -1)

        trainer.logger.experiment.log({
            "examples": [wandb.Image(x, caption=f"Pred:{pred}, Label:{y}") 
                            for x, pred, y in zip(val_imgs, preds, self.val_labels)],
            "global_step": trainer.global_step
            })

In [37]:
class MNISTDataModule(pl.LightningDataModule):

    def __init__(self, data_dir='./', batch_size=128):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))])

    def prepare_data(self):
        # download data, train then test
        MNIST(self.data_dir, train=True, download=True)
        MNIST(self.data_dir, train=False, download=True)

    def setup(self, stage=None):

        # we set up only relevant datasets when stage is specified
        if stage == 'fit' or stage is None:
            mnist = MNIST(self.data_dir, train=True, download=True, transform=self.transform)
            self.mnist_train, self.mnist_val = random_split(mnist, [55000, 5000])
        if stage == 'test' or stage is None:
            self.mnist_test = MNIST(self.data_dir, train=False, download=True, transform=self.transform)

    # we define a separate DataLoader for each of train/val/test
    def train_dataloader(self):
        mnist_train = DataLoader(self.mnist_train, batch_size=self.batch_size)
        return mnist_train

    def val_dataloader(self):
        mnist_val = DataLoader(self.mnist_val, batch_size=10 * self.batch_size)
        return mnist_val

    def test_dataloader(self):
        mnist_test = DataLoader(self.mnist_test, batch_size=10 * self.batch_size)
        return mnist_test

# setup data
mnist = MNISTDataModule()
mnist.setup()

# grab samples to log predictions on
samples = next(iter(mnist.val_dataloader()))

In [38]:
wandb_logger = WandbLogger(project="lit-wandb")

In [39]:
trainer = pl.Trainer(
    progress_bar_refresh_rate=20,
    logger=wandb_logger,    # W&B integration
    log_every_n_steps=50,   # set the logging frequency
    gpus=(1 if torch.cuda.is_available() else 0),
    max_epochs=10,           # number of epochs
    deterministic=True,     # keep it deterministic
    callbacks=[ImagePredictionLogger(samples)] # see Callbacks section
    )

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [40]:
# fit the model
trainer.fit(model, mnist)

# evaluate the model on a test set
trainer.test(datamodule=mnist,
             ckpt_path=None)  # uses last-saved model

wandb.finish()


  | Name      | Type     | Params
---------------------------------------
0 | layer_1   | Linear   | 100 K 
1 | layer_2   | Linear   | 33.0 K
2 | layer_3   | Linear   | 2.6 K 
3 | train_acc | Accuracy | 0     
4 | valid_acc | Accuracy | 0     
5 | test_acc  | Accuracy | 0     
---------------------------------------
136 K     Trainable params
0         Non-trainable params
136 K     Total params


wandb: WARNING Step must only increase in log calls.  Step 2149 < 2151; dropping {'valid/loss_epoch': 0.23160718381404877, 'valid/acc_epoch': 0.9330000281333923, 'epoch': 4}.
wandb: WARNING Step must only increase in log calls.  Step 2149 < 2151; dropping {'train/loss_epoch': 0.22832168638706207, 'train/acc_epoch': 0.9340000152587891, 'epoch': 4}.


wandb: WARNING Step must only increase in log calls.  Step 4299 < 4301; dropping {'valid/loss_epoch': 0.21487748622894287, 'valid/acc_epoch': 0.9386000037193298, 'epoch': 9}.
wandb: WARNING Step must only increase in log calls.  Step 4299 < 4301; dropping {'train/loss_epoch': 0.2083483338356018, 'train/acc_epoch': 0.9397818446159363, 'epoch': 9}.



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test/acc_epoch': tensor(0.9385, device='cuda:0'),
 'test/loss_epoch': tensor(0.2078, device='cuda:0'),
 'valid/acc_epoch': tensor(0.9386, device='cuda:0'),
 'valid/loss_epoch': tensor(0.2149, device='cuda:0')}
--------------------------------------------------------------------------------


global_step,4299
_step,8601
_runtime,135
_timestamp,1608010084
train/loss_step,0.13611
train/acc_step,0.97727
valid/loss_epoch,0.21803
valid/acc_epoch,0.9378
epoch,9
train/loss_epoch,0.21208
train/acc_epoch,0.93884


global_step,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▆▆▇▇▇▇██
_step,▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃▃▃▄▄▄▄▄▄▄▄▄▄▄█
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/loss_step,▃▅▂▂▃▆▄▂▃▄▆▃▄▃█▃▃▃▃▄▃▄▂▃▃▄▄▂▂▂▄▃▃▃▃▁▂▂▃▄
train/acc_step,▅▃▅▅▅▃▃▅▃▅▃▃▃▄▁▅▅▅▄▃▆▃▅▃▅▃▃▆▅▆▃▅▃▅▃█▆▅▅▄
valid/loss_epoch,█▇▆▅▃▃▂▁
valid/acc_epoch,▁▂▃▄▅▆▇█
epoch,▁▂▃▃▅▆▆▇█
train/loss_epoch,█▇▆▅▃▃▂▁
train/acc_epoch,▁▂▃▄▆▆▇█
